In [1]:
from mpl_toolkits import mplot3d
%matplotlib qt5
import numpy as np
import random
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
from mpl_toolkits.mplot3d.art3d import Poly3DCollection, Line3DCollection
from matplotlib.widgets import Button, Slider

# массив вершин призмы
v = np.array([[30, 30, 0], [30, 80, 0], [70, 80, 0],  [70, 30, 0], [30, 30, 150], [30, 80, 150], [70, 80, 90],  [70, 30, 90]])

# массив граней призмы
verts = [[v[0],v[1],v[5],v[4]], [v[0],v[3],v[7],v[4]], [v[2],v[3],v[7],v[6]], [v[1],v[2],v[6],v[5]], [v[0],v[1],v[2],v[3]], [v[4],v[5],v[6],v[7]]]

# массив цветов для блоков разбиения
colors = ['blue', 'orange', 'green', 'red', 'purple', 'yellow', 'brown', 'pink', 'gray', 'olive', 'cyan', 'magenta', 'salmon', 'lawngreen', 'midnightblue']

# начальные параметры количества разбиений и времени разбиения
numOfSplits = 5
Time = 0.1

fig = plt.figure()

# полноэкранный режим
plt.get_current_fig_manager().window.showMaximized()

# призма
ax = fig.add_axes([-0.05, 0, 0.775, 0.99999], projection = '3d') 
ax.scatter3D(v[:, 0], v[:, 1], v[:, 2])

# установка положения осей
ax.view_init(azim = -60, elev = 49)

def paramsOfPrism():
    # параметры призмы
    collection=Poly3DCollection(verts, linewidths = 5, alpha = .15)
    collection.set_facecolor('cyan')
    collection.set_edgecolor('black')
    ax.add_collection3d(collection)

    # установка размеров осей
    ax.set_zlim(0, 130)
    ax.set_xlim(0, 100)
    ax.set_ylim(0, 100)

    # подписи осей
    ax.set_zlabel('z')
    ax.set_xlabel('x')
    ax.set_ylabel('y')

paramsOfPrism()

# параметры графика сечений
ax1 = fig.add_axes([0.75, 0.5, 0.2, 0.35])
ax1.set_xlim(left = 0, right = 100)
ax1.set_ylim(bottom = 0, top = 100)

# слайдер №1
slider1Ax = fig.add_axes([0.75, 0.3, 0.2, 0.02])
slider1 = Slider(slider1Ax, 'Количество разбиений', valmin = 5, valmax = 100, valinit = 5, valstep = 5)

def update1(val):
    global numOfSplits
    numOfSplits = slider1.val
    
slider1.on_changed(update1)

# слайдер №2
slider2Ax = fig.add_axes([0.75, 0.2, 0.2, 0.02])
slider2 = Slider(slider2Ax, 'Время разбиения', valmin = 0.01, valmax = 1, valinit = 0.1)

def update2(val):
    global Time
    Time = slider2.val
    
slider2.on_changed(update2)

# кнопка запуска разбиений
buttonAx  = fig.add_axes([0.75, 0.05, 0.2, 0.05])
button = Button(buttonAx, 'Запустить разбиение')

# рисование блоков разбиений
def drawing(b, t, i):
    global ax, ax1, Time
    
    # очистка графика сечений и установка параметров
    ax1.clear()
    ax1.set_xlim(left = 0, right = 100)
    ax1.set_ylim(bottom = 0, top = 100)
    ax1.fill(b[:, 0], b[:, 1], colors[i % 15])
    
    # массив граней блока для разбиения
    planes = [[b[0],b[1],b[2],b[3]],
             [t[0],t[1],t[2],t[3]],
             [b[0],t[0],t[1],b[1]],
             [b[1],t[1],t[2],b[2]],
             [b[2],t[2],t[3],b[3]],
             [b[3],t[3],t[0],b[0]]]
    
    # установка параметров блока для разбиения
    addplane = Poly3DCollection(planes, linewidths = 1, alpha = .8)
    addplane.set_facecolor(colors[i % 15])
    addplane.set_edgecolor('black')
    ax.add_collection3d(addplane)
    
    # запись значений z
    ax.set_zlabel('z = ' + str(i * 150 / numOfSplits))
    
    plt.draw()
    plt.pause(Time)
    
# обработка нажатия на кнопку разбиения
def Click(event):
    global numOfSplits, Speed, v, verts, colors
    
    # очистка фигуры и блоков
    ax1.clear()
    ax.clear()
    
    ax.scatter3D(v[:, 0], v[:, 1], v[:, 2])
    paramsOfPrism();
    
    # разбиение нижней части призмы
    for i in range(int(3 * numOfSplits / 5)):
        
        # нижняя грань блока разбиения
        b = np.array([v[x] + [0, 0, 150 * i / numOfSplits] for x in range(4)])
        
        # верхняя грань блока разбиения
        t = np.array([b[x] + [0, 0, 150 / numOfSplits] for x in range(4)])
        
        drawing(b, t, i)
    
    # разбиение верхней части призмы
    for i in range(int(3 * numOfSplits / 5), int(numOfSplits)):
        
        w0 = i / numOfSplits
        # нижняя грань блока разбиения
        b = np.array([v[0] + [0, 0, 150 * w0],
                      v[1] + [0, 0, 150 * w0],
                      v[1] + [100*(1 - w0), 0, 150 * w0],
                      v[0] + [100*(1 - w0), 0, 150 * w0]])
        
        # верхняя грань блока разбиения
        t = np.array([b[x] + [0, 0, 150 / numOfSplits] for x in range(4)])
        
        drawing(b, t, i)

button.on_clicked(Click)

0